Youcef Boulfrad

Master STD

In [47]:
# Partie 1 : Configuration initiale 

## Installation des bibliothèques requises
!pip install requests beautifulsoup4

## Création de la structure du projet

# En bash : 
# mkdir -p Web_Crawler
# cd Web_Crawler

# Ouverture du dossier avec VSCODE

# Création d'un Repo GitHub distant, et local, et utilisation des commandes GIT pour le mettre à jour

## Importation des modules nécessaires
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import urllib.robotparser
import json
import re

## Implémentation des fonctions de base pour les requêtes HTTP
def telecharger_page(url):
    """Télécharge le contenu HTML d'une URL donnée."""
    try:
        en_tetes = {"User-Agent": "Explorateur-ENSAI"}
        reponse = requests.get(url, headers=en_tetes, timeout=5)
        reponse.raise_for_status()
        return reponse.text
    except requests.RequestException as e: # pour lever une exception en cas d'échec du téléchargement
        print(f" Échec du téléchargement {url}: {e}")
        return None

## Ajout de la notion de politesse
def attendre():
    """Ajoute un délai pour éviter de surcharger le serveur (politesse)."""
    time.sleep(1)  # Attente de 1 seconde entre chaque requête

print("Configuration initiale terminée !")

Defaulting to user installation because normal site-packages is not writeable
Configuration initiale terminée !


In [49]:
# Partie 2 : Extraction du contenu d'une page HTML

## Vérification des droits de parsing via robots.txt (pour une URL donnée)
def autorise_crawl(url):
    """Vérifie si le crawler a le droit de parser une page selon robots.txt."""
    robot_parser = urllib.robotparser.RobotFileParser()
    domaine = "{uri.scheme}://{uri.netloc}".format(uri=urlparse(url))
    robots_url = urljoin(domaine, "/robots.txt")
    robot_parser.set_url(robots_url)
    try:
        robot_parser.read()
        return robot_parser.can_fetch("*", url)
    except:                           # lève une exception sur l'URL n'autorise pas l'extraction de contenu
        print(f"Impossible de lire robots.txt sur {robots_url}")
        return False

## Fonction pour parser le HTML (c'est à dire extraire les principales informations d'une page HTML)
def parser_html(html, url):
    """Parse le contenu HTML et extrait titre, premier paragraphe et liens."""
    soupe = BeautifulSoup(html, "html.parser")
    
    # Extraction du titre
    titre = soupe.title.string.strip() if soupe.title else "Sans titre"
    
    # Extraction du premier paragraphe
    premier_paragraphe = ""
    paragraphes = soupe.find_all("p")
    if paragraphes:
        premier_paragraphe = paragraphes[0].get_text().strip()
    
    # Extraction des liens internes et source
    domaine = "{uri.scheme}://{uri.netloc}".format(uri=urlparse(url))
    liens = {}
    for balise_a in soupe.find_all("a", href=True):
        lien = urljoin(url, balise_a["href"])
        if domaine in lien:
            liens[lien] = url  # Stocker la source du lien
    
    return {"titre": titre, "url": url, "premier_paragraphe": premier_paragraphe, "liens": liens}

print("Extraction du contenu prête à être utilisée !")

Extraction du contenu prête à être utilisée !


In [50]:
# Parties 3 et 4 : Logique de crawling et stockage des URL crawlées

class Crawler:
    def __init__(self, url_depart, max_pages=50): #Arrêt à 50 pages
        self.url_depart = url_depart
        self.max_pages = max_pages
        self.pages_visitees = set()
        self.pages_a_visiter = [url_depart]
        self.resultats = []

    def explorer(self):
        """Parcourt les pages en suivant les liens, priorisant ceux contenant 'product'."""
        while self.pages_a_visiter and len(self.pages_visitees) < self.max_pages:
            url = self.pages_a_visiter.pop(0)
            if url in self.pages_visitees or not autorise_crawl(url):  # Ne pas pop les URL n'autorisant pas le crawl, ni celles déjà visitées
                continue
            print(f"🔍 Exploration de: {url}")
            html = telecharger_page(url)
            if not html:                            # Ne pas parser les pages non écrites en HTML
                continue
            contenu = parser_html(html, url)
            self.resultats.append(contenu)
            self.pages_visitees.add(url)
            
            # Ajout des nouveaux liens en priorisant ceux qui contiennent 'product'
            nouveaux_liens = sorted(contenu["liens"].keys(), key=lambda x: "product" not in x)
            self.pages_a_visiter.extend(nouveaux_liens)
            
            attendre()
        self.sauvegarder_resultats()
    
    
    # Ici, pour crawler à partir de plusieurs URL de départ différentes, on stocke les sorties JSON sous des noms différents, en utilisant la bibliothèque "re"
    def sauvegarder_resultats(self):
        """Sauvegarde des résultats dans un fichier JSON unique pour chaque URL de départ."""
        identifiant_unique = re.sub(r'[^a-zA-Z0-9]', '_', self.url_depart)[:50]  # Nettoyage et limitation de la longueur
        nom_fichier = f"resultats_{identifiant_unique}.json"
        with open(nom_fichier, "w", encoding="utf-8") as fichier:
            json.dump(self.resultats, fichier, indent=4, ensure_ascii=False)
        print(f"Exploration terminée ! Résultats sauvegardés dans '{nom_fichier}'.")

print("Logique de crawling implémentée avec stockage des résultats spécifique à chaque URL de départ !")

Logique de crawling implémentée avec stockage des résultats spécifique à chaque URL de départ !


In [52]:
# Partie 5 : Test du crawler sur différentes pages (Donnant chacune une sortie JSON spécifique)

def tester_crawler(url_depart, max_pages):
    """Test du crawler sur différentes pages de départ et gestion des erreurs courantes."""
    try:
        crawler = Crawler(url_depart, max_pages)
        crawler.explorer()
    except Exception as e:
        print(f"Erreur lors du test du crawler : {e}")
        
# Exemple d'utilisation

# Nous arrêtons à 10 liens par URL de départ pour réduire le temps d'exploration
tester_crawler("https://web-scraping.dev/products", 10)
tester_crawler("https://ensai.fr", 10)
tester_crawler("https://insee.fr", 10)
tester_crawler("https://le-recensement-et-moi.fr", 10)

# Dans VScode, actualiser le contenu du fichier Web_Crawler (à gauche) pour voir apparaitre les sorties JSON


🔍 Exploration de: https://web-scraping.dev/products
🔍 Exploration de: https://web-scraping.dev/products?category=apparel
🔍 Exploration de: https://web-scraping.dev/products?category=consumables
🔍 Exploration de: https://web-scraping.dev/products?category=household
🔍 Exploration de: https://web-scraping.dev/product/1
🔍 Exploration de: https://web-scraping.dev/product/2
🔍 Exploration de: https://web-scraping.dev/product/3
🔍 Exploration de: https://web-scraping.dev/product/4
🔍 Exploration de: https://web-scraping.dev/product/5
🔍 Exploration de: https://web-scraping.dev/products?page=1
Exploration terminée ! Résultats sauvegardés dans 'resultats_https___web_scraping_dev_products.json'.
🔍 Exploration de: https://ensai.fr
🔍 Exploration de: https://ensai.fr/
🔍 Exploration de: https://ensai.fr/en/
🔍 Exploration de: https://ensai.fr/actu-et-evenements/
🔍 Exploration de: https://ensai.fr/contactez-nous/
🔍 Exploration de: https://ensai.fr/1-ensai/grande-ecole-data-science/
🔍 Exploration de: https